In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install langchain langchain-community langchain-core transformers==4.52.4

  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.52.4-py3-none-any.whl (10.5 MB)


In [ ]:
from huggingface_hub import login
login()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

2025-11-19 21:23:59.342677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763587439.369689     217 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763587439.377623     217 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [72]:
def generate_text(prompt, max_length=1000, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs][0]

In [73]:
#define reponse schema
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
import re

full_name_schema = ResponseSchema(
    name="full_name",
    description="The full name of the person in the CV"
)
email_schema = ResponseSchema(
    name="email",
    description="The email address of the person"
)
education_schema = ResponseSchema(
    name="education",
    description="The education details such as degree, institution, and graduation year"
)
skills_schema = ResponseSchema(
    name="skills",
    description="A list of technical or soft skills mentioned in the CV"
)
experience_schema = ResponseSchema(
    name="experience",
    description="Work experience details including role, company, and years"
)

response_schemas = [full_name_schema, email_schema, education_schema, skills_schema, experience_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

In [74]:
#prompt
CV_extraction_template = """
You are an intelligent CV parser. Extract the following information from the provided resume text:

- Full name
- Email
- Education (degree, institution, year)
- Skills
- Experience (role, company, years)

Return the output in JSON format exactly as instructed below.

{format_instructions}

Resume text:
{resume_text}
"""

'\nYou are an intelligent CV parser. Extract the following information from the provided resume text:\n\n- Full name\n- Email\n- Education (degree, institution, year)\n- Skills\n- Experience (role, company, years)\n\nReturn the output in JSON format exactly as instructed below.\n\n{format_instructions}\n\nResume text:\n{resume_text}\n'

In [78]:
#load pdf and format prompt 
from langchain.document_loaders import PyPDFLoader
pdf_path = "/kaggle/input/dataset99/CV_nourmohamed.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load()
resume_text = " ".join([page.page_content for page in pages])


prompt = PromptTemplate(
    template=CV_extraction_template,
    input_variables=["resume_text", "format_instructions"],
).format(resume_text=resume_text, format_instructions=format_instructions)

In [80]:
response = generate_text(prompt, max_length=2000)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [77]:
print(response)


You are an intelligent CV parser. Extract the following information from the provided resume text:

- Full name
- Email
- Education (degree, institution, year)
- Skills
- Experience (role, company, years)

Return ONLY a valid JSON object, with all fields present. Do NOT add any explanations or markdown. 
If a field is missing, set it to an empty string or empty list.

The JSON must match exactly:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"full_name": string  // The full name of the person in the CV
	"email": string  // The email address of the person
	"education": string  // The education details such as degree, institution, and graduation year
	"skills": string  // A list of technical or soft skills mentioned in the CV
	"experience": string  // Work experience details including role, company, and years
}
```

Resume text:
Nour Mohamed  
UNDERGRADUATE SOFTWARE ENGINEER 
Egypt, Ale

In [ ]:
def extract_json_block(text):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)

    return f"```json\n{matches[-1]}\n```"

In [ ]:
json_text = extract_json_block(response)
print(json_text)

In [ ]:
output_data = output_parser.parse(json_text)
print(output_data)

In [ ]:
output_data["full_name"]

In [ ]:
pip install fastapi uvicorn pyngrok transformers==4.52.4 accelerate -q

In [5]:
NGROK_TOKEN = "35f7eKk4dud8PxL7IPbSrLXpOQB_4aEX5ygh5ZF5r7jtxZvcd"
API_KEY = "secret123"

In [6]:
pip install fastapi uvicorn pyngrok transformers==4.52.4 accelerate -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [90]:
from fastapi import FastAPI, UploadFile, File
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
import re
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# ----------- Define response schema -----------
full_name_schema = ResponseSchema(
    name="full_name",
    description="The full name of the person in the CV"
)
email_schema = ResponseSchema(
    name="email",
    description="The email address of the person"
)
education_schema = ResponseSchema(
    name="education",
    description="The education details such as degree, institution, and graduation year"
)
skills_schema = ResponseSchema(
    name="skills",
    description="A list of technical or soft skills mentioned in the CV"
)
experience_schema = ResponseSchema(
    name="experience",
    description="Work experience details including role, company, and years"
)

response_schemas = [full_name_schema, email_schema, education_schema, skills_schema, experience_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# ----------- Prompt template -----------
CV_extraction_template = """
You are an intelligent CV parser. Extract the following information from the provided resume text:

- Full name
- Email
- Education (degree, institution, year)
- Skills
- Experience (role, company, years)

Return ONLY a valid JSON object, with all fields present. Do NOT add any explanations or markdown. 
If a field is missing, set it to an empty string or empty list.

The JSON must match exactly:
{format_instructions}

Resume text:
{resume_text}
"""

# ----------- Helper functions -----------
def extract_json_block(text):
    pattern = r'```json\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)

    return f"```json\n{matches[-1]}\n```"

def generate_text(prompt: str, max_length=2000, num_return_sequences=1) -> str:
    """Generate text using the model."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id  # avoids generation errors
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ----------- FastAPI app -----------
app = FastAPI(title="CV Extraction API", description="Extract structured data from resumes using Mistral-Nemo")

@app.post("/extract")
async def extract_cv_data(file: UploadFile = File(...)):
    # ----------- Save uploaded PDF temporarily -----------
    contents = await file.read()
    temp_path = "/tmp/temp_resume.pdf"
    with open(temp_path, "wb") as f:
        f.write(contents)

    # ----------- Load PDF content -----------
    loader = PyPDFLoader(temp_path)
    pages = loader.load()
    resume_text = " ".join([page.page_content for page in pages])

    # ----------- Prepare prompt -----------
    prompt = PromptTemplate(
        template=CV_extraction_template,
        input_variables=["resume_text", "format_instructions"],
    ).format(resume_text=resume_text, format_instructions=format_instructions)

    # ----------- Generate model output -----------
    response_text = generate_text(prompt)

    # ----------- Extract JSON block and parse safely -----------
    json_text = extract_json_block(response_text)
    try:
        output_data = output_parser.parse(json_text)
    except Exception:
        try:
            output_data = json.loads(json_text)
        except:
            # Fallback to empty structured response
            output_data = {
                "full_name": "",
                "email": "",
                "education": "",
                "skills": [],
                "experience": []
            }

    return output_data


In [93]:
import uvicorn, threading, time, socket
from pyngrok import ngrok, conf
from fastapi import FastAPI, Request, HTTPException
def free_port():
    s = socket.socket()
    s.bind(('', 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = free_port()
conf.get_default().auth_token = NGROK_TOKEN #put token of ngrok
public_url = ngrok.connect(port).public_url
print("Your public URL:", public_url)

def run(): uvicorn.run(app, host="0.0.0.0", port=port)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)

Your public URL: https://apprehensibly-cresylic-layla.ngrok-free.dev


INFO:     Started server process [217]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:41503 (Press CTRL+C to quit)


In [94]:
import requests

URL = "https://apprehensibly-cresylic-layla.ngrok-free.dev/extract"

# Open the PDF file in binary mode
with open("/kaggle/input/dataset99/CV_nourmohamed.pdf", "rb") as f:
    files = {"file": ("CV_nourmohamed.pdf", f, "application/pdf")}
    
    # Send POST request with multipart/form-data
    response = requests.post(URL, files=files)

# Handle response
if response.status_code == 200:
    print("Extracted Data:", response.json())
else:
    print("Error:", response.status_code, response.text)


INFO:     34.171.103.41:0 - "POST /extract HTTP/1.1" 200 OK
Extracted Data: {'full_name': 'Nour Mohamed', 'email': 'nour.muhammed120@gmail.com', 'education': 'Bachelor’s Degree of Software industry and multimedia, Alexandria University, 2022 - 2026', 'skills': ['Python', 'Java', 'JavaScript', 'Dart', 'SQL', 'powershell', 'JavaScript', 'linux(ubunto)', 'Flutter', 'Android fundamentals', 'responsive UI design', 'HTML', 'CSS', 'modern UI frameworks', 'Arrays', 'trees', 'graphs', 'sorting/searching algorithms', 'Class design', 'inheritance', 'encapsulation', 'polymorphism', 'Pandas', 'NumPy', 'Flutter SDK', 'Problem-solving', 'teamwork', 'presentation skills', 'effective communication', 'eager to learn', 'motivated', 'detail-oriented'], 'experience': [{'role': 'Business Intelligence Development Trainee', 'company': 'Commercial International Bank Summer Training (CIB)', 'years': 'July 2023 – Augst2023'}, {'role': 'Data Scientist and AI Trainee', 'company': 'Digital Egypt Pioneers Initiative

In [92]:
from pyngrok import ngrok

ngrok.kill()  # kills all active tunnels
